In [61]:
import torch
from torchvision.transforms import v2
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.io import decode_image
from pathlib import Path
device = torch.device('cuda')

In [62]:
PLANT_CLASSES = [
    'Black-grass',
    'Charlock', 
    'Cleavers',
    'Common Chickweed',
    'Common wheat',
    'Fat Hen',
    'Loose Silky-bent',
    'Maize',
    'Scentless Mayweed',
    'Shepherds Purse',
    'Small-flowered Cranesbill',
    'Sugar beet'
]

class PlantDataset(Dataset):

    def __init__(self, rootDir, transform=None):
        self.rootDir = Path(rootDir)
        self.transforms = transform
        self.classes = PLANT_CLASSES
        self.classsToidx = {plant : idx for idx, plant in enumerate(PLANT_CLASSES)} #將classString轉成 index 的 dict

        self.samples = []
        self._load_samples()
    
    def _load_samples(self):
        """載入指定path中的資料 將'路徑'和label紀錄到samples中"""
        for class_name in self.classes:
            class_dir = self.rootDir / class_name
            if class_dir.exists():
                for img_path in class_dir.glob('*.png'):
                    self.samples.append((str(img_path), self.classsToidx[class_name]))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        img_path, label = self.samples[index]
        try:
            image = decode_image(img_path, mode='RGB')
        except Exception as e:
            print(f"無法載入圖片{img_path}:{e}")
        if self.transforms:
            image = self.transforms(image)
        return image, label

In [63]:
transform_train = v2.Compose([
    v2.Resize((256, 256)),
    v2.RandomResizedCrop(224), #resnet50標準輸入size
    v2.RandomHorizontalFlip(0.5),
    v2.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1
    ),

    v2.RandomRotation(15),
    v2.RandomGrayscale(p=0.1),

    v2.ToDtype(torch.float32, scale=True),

    v2.Normalize(
        mean=[0.485, 0.456, 0.406],#0~1 float計算的mean
        std=[0.229, 0.224, 0.225]
    )#ImageNet 資料的mean 跟 std
])

transform_val = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(
        mean=[0.485, 0.456, 0.406],#0~1 float計算的mean
        std=[0.229, 0.224, 0.225]
    )#ImageNet 資料的mean 跟 std
])

In [64]:
def create_dataloaders_with_split(data_dir, batch_size=64, num_workers=2, val_split=0.2):

    train_dir = data_dir / 'train'

    full_dataset = PlantDataset(
        rootDir=train_dir,
        transform=None
    )

    total_size = len(full_dataset)
    val_size = int(total_size * val_split)
    train_size = total_size - val_size

    train_indices, val_indices = random_split(
        #為了套不同的transfroms 用idx分割
        range(total_size), 
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(full_dataset, val_indices)
    
    train_dataset.dataset.transforms = transform_train
    val_dataset.dataset.transforms = transform_val

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True,
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False,
    )

    return train_loader, val_loader

In [65]:
class TestDataset(Dataset):
    
    def __init__(self, rootDir, transform=None):
        self.rootDir = Path(rootDir)
        self.transforms = transform
        self.samples = []
        self._load_samples()
    
    def _load_samples(self):
        """載入測試資料夾中的所有圖片"""
        for img_path in self.rootDir.glob('*.png'):
            self.samples.append(str(img_path))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        img_path = self.samples[index]
        try:
            image = decode_image(img_path, mode='RGB')
        except Exception as e:
            print(f"❌ 無法載入圖片 {img_path}: {e}") 
        
        if self.transforms:
            image = self.transforms(image)
        
        filename = Path(img_path).name
        return image, filename

def create_test_dataloader(data_dir, batch_size=64, num_workers=2):
    """建立測試資料載入器"""
    
    test_dir = data_dir / 'test'
    
    test_dataset = TestDataset(
        rootDir=test_dir,
        transform=transform_val
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False,
    )
    
    return test_loader

In [66]:
data_dir = Path('./data')
train_loader, val_loader = create_dataloaders_with_split(data_dir, batch_size=32, num_workers=0)
test_loader = create_test_dataloader(data_dir, batch_size=32, num_workers=0)

In [67]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

In [68]:
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

def create_resnet50_model(num_classes=12):
    """建立 ResNet-50 模型"""
    
    # 🎯 載入預訓練模型
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    return model

def create_resnet50_model_freeze(num_classes=12):
    """建立 ResNet-50 模型"""
    
    # 🎯 載入預訓練模型
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    
    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    return model

model = create_resnet50_model(num_classes=len(PLANT_CLASSES))
model = model.to(device)
modelFreeze = create_resnet50_model_freeze(num_classes=len(PLANT_CLASSES))
modelFreeze = modelFreeze.to(device)

# 📊 顯示模型資訊
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"📈 總參數量: {total_params:,}")
print(f"🎯 可訓練參數: {trainable_params:,}")


📈 總參數量: 23,532,620
🎯 可訓練參數: 23,532,620


In [69]:
criterion = nn.CrossEntropyLoss()
    
optimizer = Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=0.0001  # L2 正則化
)
    
scheduler = StepLR(
    optimizer,
    step_size=7,    # 每 7 個 epoch 降低學習率
    gamma=0.1       # 學習率乘以 0.1
)

In [70]:
def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """訓練一個 epoch"""
    
    model.train()  # 設定為訓練模式
    running_loss = 0.0
    correct = 0
    total = 0
    
    print(f"🚂 Epoch {epoch} 訓練中...")
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        # 📦 資料移到 GPU
        images, labels = images.to(device), labels.to(device)
        
        # 🔄 前向傳播
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 🔙 反向傳播
        loss.backward()
        optimizer.step()
        
        # 📊 統計
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # 📈 顯示進度
        if batch_idx % 10 == 0:
            print(f"  批次 {batch_idx}/{len(train_loader)}: "
                  f"Loss={loss.item():.4f}, "
                  f"Acc={100.*correct/total:.2f}%")
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_one_epoch(model, val_loader, criterion, device, epoch):
    """驗證一個 epoch"""
    
    model.eval()  # 設定為評估模式
    running_loss = 0.0
    correct = 0
    total = 0
    
    print(f"✅ Epoch {epoch} 驗證中...")
    
    with torch.no_grad():  # 不計算梯度
        for images, labels in val_loader:
            # 📦 資料移到 GPU
            images, labels = images.to(device), labels.to(device)
            
            # 🔄 前向傳播
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # 📊 統計
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


In [71]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                device, num_epochs=20, save_path='best_model.pth'):
    """完整訓練流程"""
    
    print("🚀 開始訓練 ResNet-50")
    print("="*50)
    print(f"📊 訓練集批次數: {len(train_loader)}")
    print(f"📊 驗證集批次數: {len(val_loader)}")
    print(f"⏰ 總訓練輪數: {num_epochs}")
    print("="*50)
    
    # 📈 記錄訓練歷史
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    
    for epoch in range(1, num_epochs + 1):
        print(f"\n🔄 Epoch {epoch}/{num_epochs}")
        print("-" * 30)
        
        # 🚂 訓練階段
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, epoch
        )
        
        # ✅ 驗證階段
        val_loss, val_acc = validate_one_epoch(
            model, val_loader, criterion, device, epoch
        )
        
        # 📅 更新學習率
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        
        # 📊 記錄結果
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # 📈 顯示結果
        print(f"\n📊 Epoch {epoch} 結果:")
        print(f"  🚂 訓練 - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
        print(f"  ✅ 驗證 - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")
        print(f"  📅 學習率: {current_lr:.6f}")
        
        # 💾 儲存最佳模型
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_acc': best_val_acc,
                'history': history
            }, save_path)
            print(f"  🎯 新的最佳模型！驗證準確率: {val_acc:.2f}%")
    
    print(f"\n🎉 訓練完成！")
    print(f"🏆 最佳驗證準確率: {best_val_acc:.2f}%")
    print(f"💾 模型已儲存至: {save_path}")
    
    return history

# 🚀 開始訓練！
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    num_epochs=20,
    save_path='resnet50_plant_classifier.pth'
)


🚀 開始訓練 ResNet-50
📊 訓練集批次數: 118
📊 驗證集批次數: 30
⏰ 總訓練輪數: 20

🔄 Epoch 1/20
------------------------------
🚂 Epoch 1 訓練中...
  批次 0/118: Loss=2.4780, Acc=6.25%
  批次 10/118: Loss=1.6328, Acc=38.35%
  批次 20/118: Loss=1.1615, Acc=51.49%
  批次 30/118: Loss=0.4183, Acc=58.06%
  批次 40/118: Loss=0.3005, Acc=62.88%
  批次 50/118: Loss=0.6914, Acc=65.75%
  批次 60/118: Loss=0.6175, Acc=68.14%
  批次 70/118: Loss=0.5082, Acc=70.29%
  批次 80/118: Loss=0.4150, Acc=72.03%
  批次 90/118: Loss=0.6999, Acc=73.18%
  批次 100/118: Loss=0.7601, Acc=74.01%
  批次 110/118: Loss=0.5192, Acc=74.77%
✅ Epoch 1 驗證中...

📊 Epoch 1 結果:
  🚂 訓練 - Loss: 0.7641, Acc: 75.05%
  ✅ 驗證 - Loss: 0.5962, Acc: 82.32%
  📅 學習率: 0.001000
  🎯 新的最佳模型！驗證準確率: 82.32%

🔄 Epoch 2/20
------------------------------
🚂 Epoch 2 訓練中...
  批次 0/118: Loss=0.2128, Acc=96.88%
  批次 10/118: Loss=0.3491, Acc=86.93%
  批次 20/118: Loss=0.1346, Acc=88.24%
  批次 30/118: Loss=0.1556, Acc=87.40%
  批次 40/118: Loss=0.3159, Acc=88.34%
  批次 50/118: Loss=0.1169, Acc=89.09%
  批次 60/11

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def plot_training_history(history, save_path='training_history.png'):
    """繪製訓練歷史圖表"""
    
    print("📊 繪製訓練歷史:")
    print("="*30)
    
    # 設置中文字體和風格
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS']
    plt.rcParams['axes.unicode_minus'] = False
    sns.set_style("whitegrid")
    
    # 創建 2x2 子圖
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('🌱 植物分類模型訓練結果', fontsize=16, fontweight='bold')
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # 📉 Loss 曲線
    axes[0, 0].plot(epochs, history['train_loss'], 'b-o', label='訓練 Loss', linewidth=2, markersize=4)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-s', label='驗證 Loss', linewidth=2, markersize=4)
    axes[0, 0].set_title('📉 Loss 變化曲線', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 📈 準確率曲線
    axes[0, 1].plot(epochs, history['train_acc'], 'b-o', label='訓練準確率', linewidth=2, markersize=4)
    axes[0, 1].plot(epochs, history['val_acc'], 'r-s', label='驗證準確率', linewidth=2, markersize=4)
    axes[0, 1].set_title('📈 準確率變化曲線', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('準確率 (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 📊 最終幾個 epoch 的詳細比較
    last_n = min(5, len(epochs))
    last_epochs = epochs[-last_n:]
    
    axes[1, 0].bar([f'E{e}' for e in last_epochs], 
                   [history['train_acc'][e-1] for e in last_epochs], 
                   alpha=0.7, color='skyblue', label='訓練')
    axes[1, 0].bar([f'E{e}' for e in last_epochs], 
                   [history['val_acc'][e-1] for e in last_epochs], 
                   alpha=0.7, color='lightcoral', label='驗證')
    axes[1, 0].set_title(f'📊 最後 {last_n} 個 Epoch 準確率比較', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('準確率 (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 📈 學習曲線分析
    train_val_gap = [abs(t - v) for t, v in zip(history['train_acc'], history['val_acc'])]
    axes[1, 1].plot(epochs, train_val_gap, 'g-^', label='訓練-驗證差距', linewidth=2, markersize=4)
    axes[1, 1].axhline(y=5, color='orange', linestyle='--', alpha=0.7, label='理想差距 (5%)')
    axes[1, 1].set_title('📈 過擬合分析', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('準確率差距 (%)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    # 📊 打印統計信息
    print(f"📊 訓練統計:")
    print(f"   最佳訓練準確率: {max(history['train_acc']):.2f}%")
    print(f"   最佳驗證準確率: {max(history['val_acc']):.2f}%")
    print(f"   最終訓練準確率: {history['train_acc'][-1]:.2f}%")
    print(f"   最終驗證準確率: {history['val_acc'][-1]:.2f}%")
    print(f"   最終過擬合程度: {abs(history['train_acc'][-1] - history['val_acc'][-1]):.2f}%")

# 🎨 繪製訓練歷史
plot_training_history(history)


In [78]:
import pandas as pd
from pathlib import Path
import torch
from tqdm import tqdm

def predict_test_dataset(model, test_loader, device, class_names, save_path='predictions.csv'):
    """對測試資料集進行預測並輸出 CSV"""
    
    print("🔍 開始預測測試資料集:")
    print("="*40)
    print(f"📊 測試批次數: {len(test_loader)}")
    print(f"📁 預測結果將保存至: {save_path}")
    print("="*40)
    
    model.eval()
    
    # 儲存預測結果
    predictions_list = []
    filenames_list = []
    confidences_list = []
    
    with torch.no_grad():
        # 使用 tqdm 顯示進度條
        for batch_idx, (images, filenames) in enumerate(tqdm(test_loader, desc="🚀 預測中")):
            images = images.to(device)
            
            # 前向傳播
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidences, predicted = torch.max(probabilities, 1)
            
            # 轉換為 CPU 並收集結果
            predicted = predicted.cpu().numpy()
            confidences = confidences.cpu().numpy()
            
            # 將預測類別索引轉換為類別名稱
            for i, (pred_idx, conf, filename) in enumerate(zip(predicted, confidences, filenames)):
                species_name = class_names[pred_idx]
                
                predictions_list.append(species_name)
                filenames_list.append(filename)
                confidences_list.append(conf)
    
    # 建立 DataFrame
    results_df = pd.DataFrame({
        'file': filenames_list,
        'species': predictions_list,
        'confidence': confidences_list
    })
    
    # 按檔名排序
    results_df = results_df.sort_values('file').reset_index(drop=True)
    
    # 保存 CSV（只包含 file 和 species 欄位，符合你的格式要求）
    final_df = results_df[['file', 'species']]
    final_df.to_csv(save_path, index=False)
    
    print(f"\n✅ 預測完成！")
    print(f"📊 總預測樣本數: {len(results_df)}")
    print(f"📁 結果已保存至: {save_path}")
    
    # 顯示預測統計
    print(f"\n📊 預測統計:")
    species_counts = results_df['species'].value_counts()
    for species, count in species_counts.items():
        percentage = (count / len(results_df)) * 100
        print(f"   {species}: {count} 張 ({percentage:.1f}%)")
    
    # 顯示信心度統計
    print(f"\n🎯 預測信心度統計:")
    print(f"   平均信心度: {results_df['confidence'].mean():.3f}")
    print(f"   最高信心度: {results_df['confidence'].max():.3f}")
    print(f"   最低信心度: {results_df['confidence'].min():.3f}")
    print(f"   高信心度 (>0.9): {(results_df['confidence'] > 0.9).sum()} 張")
    print(f"   低信心度 (<0.5): {(results_df['confidence'] < 0.5).sum()} 張")
    
    # 顯示前幾個預測結果作為範例
    print(f"\n📋 前 10 個預測結果:")
    print(final_df.head(10).to_string(index=False))
    
    return results_df, final_df

# 🚀 執行測試預測
def run_test_prediction():
    """執行完整的測試預測流程"""
    
    print("🚀 開始測試預測流程:")
    print("="*50)
    
    # 1. 載入最佳模型
    model_path = 'resnet50_plant_classifier.pth'
    print(f"📥 載入模型: {model_path}")
    
    try:
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ 模型載入成功! (最佳驗證準確率: {checkpoint['best_val_acc']:.2f}%)")
    except Exception as e:
        print(f"❌ 模型載入失敗: {e}")
        return None
    
    
    if len(test_loader) == 0:
        print("❌ 測試資料夾為空或無法讀取")
        return None
    
    print(f"📊 測試資料載入完成: {len(test_loader.dataset)} 張圖片")
    
    # 3. 進行預測
    results_df, final_df = predict_test_dataset(
        model=model,
        test_loader=test_loader,
        device=device,
        class_names=PLANT_CLASSES,
        save_path='test_predictions.csv'
    )
    
    return results_df, final_df

# 🎯 執行預測
prediction_results = run_test_prediction()


🚀 開始測試預測流程:
📥 載入模型: resnet50_plant_classifier.pth
✅ 模型載入成功! (最佳驗證準確率: 89.89%)
📊 測試資料載入完成: 794 張圖片
🔍 開始預測測試資料集:
📊 測試批次數: 25
📁 預測結果將保存至: test_predictions.csv


🚀 預測中: 100%|██████████| 25/25 [00:06<00:00,  3.71it/s]


✅ 預測完成！
📊 總預測樣本數: 794
📁 結果已保存至: test_predictions.csv

📊 預測統計:
   Loose Silky-bent: 145 張 (18.3%)
   Common Chickweed: 99 張 (12.5%)
   Small-flowered Cranesbill: 85 張 (10.7%)
   Scentless Mayweed: 82 張 (10.3%)
   Sugar beet: 72 張 (9.1%)
   Fat Hen: 66 張 (8.3%)
   Charlock: 64 張 (8.1%)
   Cleavers: 48 張 (6.0%)
   Shepherds Purse: 45 張 (5.7%)
   Common wheat: 38 張 (4.8%)
   Maize: 36 張 (4.5%)
   Black-grass: 14 張 (1.8%)

🎯 預測信心度統計:
   平均信心度: 0.910
   最高信心度: 1.000
   最低信心度: 0.310
   高信心度 (>0.9): 588 張
   低信心度 (<0.5): 17 張

📋 前 10 個預測結果:
         file                   species
0021e90e4.png Small-flowered Cranesbill
003d61042.png                   Fat Hen
007b3da8b.png                Sugar beet
0086a6340.png          Common Chickweed
00c47e980.png                Sugar beet
00d090cde.png          Loose Silky-bent
00ef713a8.png          Common Chickweed
01291174f.png                   Fat Hen
026716f9b.png          Loose Silky-bent
02cfeb38d.png          Loose Silky-bent
